# Chatterbox TTS - Colab Notebook
This notebook demonstrates how to use Chatterbox, a state-of-the-art open-source TTS model. Follow the steps below to get started.

## 1. Install Dependencies
First, let's install the required packages.

In [ ]:
!pip install chatterbox-tts

## 2. Import Libraries and Setup

In [ ]:
import torchaudio as ta
import torch
from chatterbox.tts import ChatterboxTTS

# Automatically detect the best available device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using device: {device}")

## 3. Load the Model
Now we'll load the pre-trained Chatterbox TTS model.

In [ ]:
model = ChatterboxTTS.from_pretrained(device=device)

## 4. Generate Speech from Text
Let's generate some speech using the default voice.

In [ ]:
text = "Hello! This is a test of the Chatterbox TTS system. It can generate natural-sounding speech from text."
wav = model.generate(text)

# Save the generated audio
output_path = "test_output.wav"
ta.save(output_path, wav, model.sr)

# Display audio player in the notebook
from IPython.display import Audio
Audio(output_path)

## 5. Voice Cloning (Optional)
You can also clone a voice by providing an audio prompt. Upload your audio file and specify its path below.

In [ ]:
# Upload your audio file and set its path here
AUDIO_PROMPT_PATH = "path_to_your_audio.wav"  # Replace with your audio file path

# Generate speech with the voice from the audio prompt
text = "This is the same text but spoken in a different voice."
wav = model.generate(text, audio_prompt_path=AUDIO_PROMPT_PATH)

# Save and play the generated audio
output_path_cloned = "test_output_cloned.wav"
ta.save(output_path_cloned, wav, model.sr)
Audio(output_path_cloned)